In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Enable this for some more verbose info.
debugging = False
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [0]:
cd /content/gdrive/My Drive/Mask_RCNN-master/fish_project

/content/gdrive/My Drive/Mask_RCNN-master/fish_project


In [0]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import random
import collections


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
ROOT_IMAGE_DIR = os.path.abspath("images/")

# Path to the dataset (note this is a shared images directory)
dataset_path = os.path.join(ROOT_IMAGE_DIR, "fish")

models_dir = os.path.join(ROOT_DIR,  "fish_project/models/")

print("base dataset dir:", dataset_path)
print("base models dir:", models_dir)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.model import log

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(models_dir, "mask_rcnn_fish_0070.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



base dataset dir: /content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/fish
base models dir: /content/gdrive/My Drive/Mask_RCNN-master/fish_project/models/


In [0]:
# for visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

%matplotlib inline

# For Config and Dataset 
sys.path.append(os.path.join(ROOT_DIR, "fish_project/"))  # To find local version
import fish_project.det as det

In [0]:
# The imgaug library is pretty flexible and make different types of augmentation possible. 
# The deterministic setting is used because any spatial changes to the image must also be 
# done to the mask. There are also some augmentors that are unsafe to apply. From the mrcnn
# library: 
# Augmentors that are safe to apply to masks: 
# ["Sequential", "SomeOf", "OneOf", "Sometimes","Fliplr", 
# "Flipud", "CropAndPad", "Affine", "PiecewiseAffine"]
# Affine, has settings that are unsafe, so always
# test your augmentation on masks

import imgaug as ia
from imgaug import augmenters as iaa

ia.seed(1)

# http://imgaug.readthedocs.io/en/latest/source/augmenters.html#sequential
seq_of_aug = iaa.Sequential([
    iaa.Crop(percent=(0, 0.1)), # random crops
    
    # horizontally flip 50% of the images
    iaa.Fliplr(0.5), 

    # Gaussian blur to 50% of the images
    # with random sigma between 0 and 0.5.
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    
    # Apply affine transformations to each image.
    # Scale/zoom them from 90% 5o 110%
    # Translate/move them, rotate them
    # Shear them slightly -2 to 2 degrees.
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-5, 5),
        shear=(-2, 2)
    )
], random_order=True) # apply augmenters in random order



In [0]:
# Create an object to hold configuration
config = det.DetConfig('Fish', ['Fish'])
# Load dataset
## annotations파일을 불러와서 클래스 구분되는 이름을 가져온다.
dataset_train, dataset_val = det.create_datasets(dataset_path+"/train",  config) 


Counter({'Fish': 106})
Counter({'Fish': 26})
Adding class   1:Fish
Adding class   1:Fish


In [0]:
# Must call before using the dataset, amount other things,
# this maps the sources to the class ids.
dataset_train.prepare()
dataset_val.prepare()

# Print a summary of the dataset and classes
print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Image Count: {}".format(len(dataset_val.image_ids)))

Image Count: 100
Image Count: 25


In [0]:
result_from_annotations = {}

In [0]:
for i in range(len(dataset_train.image_ids)):
    result_from_annotations[dataset_train.image_info[i]['id']] =\
        dataset_train.load_mask(i)[0]
    
result_from_annotations

{'20170420_231814.jpg': array([[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        ...,
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]]]), '20170422_123719.jpg': array([[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ..

In [0]:
for i in range(len(dataset_val.image_ids)):
    result_from_annotations[dataset_val.image_info[i]['id']] =\
        dataset_val.load_mask(i)[0]

In [0]:
## 마스크 결과 불러오기

import pickle 
file=open("result_masks","rb") 
result_masks=pickle.load(file) 
file.close() 

In [0]:
masks = dict(result_masks)

In [0]:
result = {**masks, **result_from_annotations}
result

{'A73EGS~P_1.jpg': array([[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        ...,
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]]]), 'A73EGS~P_2.jpg': array([[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],
 
        [[False],
         [False],
         [False],
         ...,
       

In [0]:
len(result)

4462

In [0]:
## 마스크 결과 저장
import pickle 

file=open("mask_shape","wb") 
pickle.dump(result, file) 
file.close()


In [0]:
## 마스크 결과 불러오기

import pickle 
file=open("mask_shape","rb") 
result=pickle.load(file) 
file.close() 

In [0]:
len(result)

4462

In [0]:
## 사진들 경로 설정
image_dir = os.path.join(ROOT_IMAGE_DIR, "raw_images")
print("image directory : ", image_dir)

image directory :  /content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/raw_images


In [0]:
config = det.DetConfig('fish', ['Fish'])
config.BATCH_SIZE = 1
config.IMAGES_PER_GPU = 1
config.display()


Configurations:
ACTIVATION                     leakyrelu
ALL_CLASS_NAMES                ['BG', 'Fish']
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CLASS_NAMES                    ['Fish']
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0,

In [0]:
# Create the model
model = modellib.MaskRCNN(mode='training', 
                          config=config,
                          model_dir=models_dir)

model.keras_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
_____________________________________

In [0]:
weights_path = os.path.join(models_dir, "fish20190227T0140/mask_rcnn_fish_0065.h5")
print("loading...", end='')
model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])
print("loaded.")

loading...Re-starting from epoch 65
loaded.


In [0]:
import imp
imp.reload(det)

<module 'fish_project.det' from '/content/gdrive/My Drive/Mask_RCNN-master/fish_project/det.py'>

In [0]:
# Create the train and val dataset.
shape_train, shape_val = det.create_datasets_total(image_dir, config, result)

# prepare datasets
shape_train.prepare()
shape_val.prepare()

Adding class   1:Fish
Adding class   1:Fish
Train list :  ['oxymonacanthus_longirostris_24.jpg', 'nemipterus_peronii_3.jpg', 'pentapodus_aureofasciatus_4.jpg', 'valamugil_seheli_4.jpg', 'lethrinus_harak_3.jpg', 'pseudolabrus_luculentus_11.jpg', 'novaculoides_macrolepidotus_7.jpg', 'seriola_hippos_2.jpg', 'scolopsis_vosmeri_4.jpg', 'lutjanus_biguttatus_4.jpg', 'halichoeres_hortulanus_4.jpg', 'coris_batuensis_17.jpg', 'meuschenia_galii_5.jpg', 'novaculoides_macrolepidotus_8.jpg', 'halichoeres_hortulanus_3.jpg', 'lethrinus_nebulosus_13.jpg', 'sardinella_gibbosa_3.jpg', 'labroides_dimidiatus_4.jpg', 'halichoeres_nigrescens_19.jpg', 'epinephelus_polyphekadion_2.jpg', 'hologymnosus_doliatus_9.jpg', 'pristipomoides_sieboldii_7.jpg', 'epinephelus_coioides_10.jpg', 'hemigymnus_fasciatus_14.jpg', 'pseudocheilinus_evanidus_1.jpg', 'anampses_meleagrides_13.jpg', 'anampses_lennardi_4.jpg', '20180227_150201.jpg', 'halichoeres_trimaculatus_2.jpg', 'lethrinus_atkinsoni_8.jpg', 'macropharyngodon_choati

In [0]:
print("Training all network")
#print("Training head layer network")
history = model.train(shape_train, shape_val,
            learning_rate=config.LEARNING_RATE,
            epochs=100,
            layers='all', 
            augmentation=seq_of_aug,
            )



Training all network

Starting at epoch 65. LR=0.001

Checkpoint Path: /content/gdrive/My Drive/Mask_RCNN-master/fish_project/models/fish20190227T0140/mask_rcnn_fish_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 66/100
100/100 [==============================] - 307s 3s/step - loss: 0.4802 - rpn_class_loss: 9.0444e-04 - rpn_bbox_loss: 0.1068 - mrcnn_class_loss: 0.0339 - mrcnn_bbox_loss: 0.1764 - mrcnn_mask_loss: 0.1621 - val_loss: 0.2780 - val_rpn_class_loss: 6.1978e-04 - val_rpn_bbox_loss: 0.1277 - val_mrcnn_class_loss: 0.0070 - val_mrcnn_bbox_loss: 0.0656 - val_mrcnn_mask_loss: 0.0771
Epoch 67/100
100/100 [==============================] - 221s 2s/step - loss: 0.2525 - rpn_class_loss: 9.6794e-04 - rpn_bbox_loss: 0.0855 - mrcnn_class_loss: 0.0069 - mrcnn_bbox_loss: 0.0627 - mrcnn_mask_loss: 0.0964 - val_loss: 0.1765 - val_rpn_class_loss: 4.4498e-04 - val_rpn_bbox_loss: 0.0635 - val_mrcnn_class_loss: 0.0032 - val_mrcnn_bbox_loss: 0.0297 - val_mrcnn_mask_loss: 0.0796
Epoch 68/100
100/100 [==============================] - 220s 2s/step - loss: 0.3235 - rpn_class_loss: 9.8330e-04 - rpn_bbox_loss: 0.1511 - mrcnn_class_loss: 0.0042 - mrcnn_bbox_loss: 0.0666 - mrcnn_mask_loss: 0.1006 - val_loss

ERROR:root:Error processing image {'id': 'gempylus_serpens_1.jpg', 'source': 'fish', 'path': '/content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/raw_images/gempylus_serpens_1.jpg'}
Traceback (most recent call last):
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1254, in load_image_gt
    hooks=imgaug.HooksImages(activator=hook))
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 407, in augment_image
    return self.augment_images([image], hooks=hooks)[0]
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 1956, in _augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_im

 75/100 [=====================>........] - ETA: 46s - loss: 0.2168 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0894 - mrcnn_class_loss: 0.0189 - mrcnn_bbox_loss: 0.0351 - mrcnn_mask_loss: 0.0722

ERROR:root:Error processing image {'id': 'gempylus_serpens_1.jpg', 'source': 'fish', 'path': '/content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/raw_images/gempylus_serpens_1.jpg'}
Traceback (most recent call last):
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1254, in load_image_gt
    hooks=imgaug.HooksImages(activator=hook))
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 407, in augment_image
    return self.augment_images([image], hooks=hooks)[0]
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 1956, in _augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_im

100/100 [==============================] - 216s 2s/step - loss: 0.2055 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0813 - mrcnn_class_loss: 0.0183 - mrcnn_bbox_loss: 0.0327 - mrcnn_mask_loss: 0.0720 - val_loss: 0.1943 - val_rpn_class_loss: 0.0012 - val_rpn_bbox_loss: 0.0574 - val_mrcnn_class_loss: 0.0134 - val_mrcnn_bbox_loss: 0.0596 - val_mrcnn_mask_loss: 0.0626
Epoch 88/100
100/100 [==============================] - 216s 2s/step - loss: 0.2126 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0781 - mrcnn_class_loss: 0.0185 - mrcnn_bbox_loss: 0.0333 - mrcnn_mask_loss: 0.0817 - val_loss: 0.1893 - val_rpn_class_loss: 8.1665e-04 - val_rpn_bbox_loss: 0.0771 - val_mrcnn_class_loss: 0.0113 - val_mrcnn_bbox_loss: 0.0372 - val_mrcnn_mask_loss: 0.0627
Epoch 89/100
100/100 [==============================] - 216s 2s/step - loss: 0.1890 - rpn_class_loss: 8.4367e-04 - rpn_bbox_loss: 0.0656 - mrcnn_class_loss: 0.0130 - mrcnn_bbox_loss: 0.0349 - mrcnn_mask_loss: 0.0746 - val_loss: 0.2278 - val_rpn_class_

In [0]:
import math

def plot_history(history):
    
    fig = plt.figure(figsize=(16,10))
       
    n_history_loss = len(history.history)
    n_epochs = len(history.epoch)
    epoch = history.epoch
    
    # The loss is in pairs, one for train, one for val
    loss_stats = [ k for k in history.history.keys() if 'val_' not in k ]
    
    n_cols = 4
    n_rows = math.ceil(len(loss_stats) / n_cols) 
    
    for i, k in enumerate(loss_stats):
        val_k = 'val_'+k
        ax = plt.subplot( n_rows, n_cols, i+1)
        ax.plot(epoch, history.history[k], label=k)
        ax.plot(epoch, history.history[val_k], label=val_k)
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Loss')
        ax.set_title(str(i)+' - '+k)
        plt.legend(shadow=True, fancybox=True)
    
    fig.tight_layout()
    plt.show()
    

In [0]:
# Plot training stats for each of the networks.
plot_history(history)

In [0]:
import glob
det_filenames = sorted(glob.glob(image_dir+'/*'))

In [0]:
## 모델을 사용해서 물고기 모양 마스크 추출

import re
from collections import defaultdict
result_masks = defaultdict()

for image_name in det_filenames:
    image = skimage.io.imread(image_name)
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    results = inf_model.detect([image], verbose=0)
    print(image_name)
    a = re.split('[/]+', image_name)
    filenames = a.pop()
    ## make dictionary to save results of masks
    result_masks[filenames] = results[0]['masks']

In [0]:
## 마스크 결과 저장
import pickle 

file=open("result_masks_2","wb") 
pickle.dump(result_masks, file) 
file.close()



In [0]:
## 마스크 결과 불러오기

import pickle 
file=open("result_masks","rb") 
result_masks=pickle.load(file) 
file.close() 

In [0]:
import re
class_ids = set()
for key in result_masks.keys():
 
    if key:
        
        ## 파일명을 class ID로 사용하기 위해서 정규식으로 잘라냄
        parse = re.sub('[0-9.]', '', key)
        parse1 = re.sub('jpg$|jpeg$', '', parse)
        parse2 = re.sub('_', ' ', count=1, string=parse1)
        parse3 = re.sub('_', '', parse2)
        class_ids.add(parse3)

        
        
        
config_id = list(class_ids)
config_id

['chascanopsetta lugubris',
 'lutjanus semicinctus',
 'pagrus auratus',
 'cephalopholis microprion',
 'pseudanthias lori',
 'aprion virescens',
 'stethojulis strigiventer',
 'lutjanus fulviflamma',
 'pentapodus paradiseus',
 'halichoeres chloropterus',
 'coris gaimard',
 'epinephelus radiatus',
 'scolopsis xenochrous',
 'macropharyngodon negrosensis',
 'alepes kleinii',
 'amanses scopas',
 'coris pictoides',
 'pseudocheilinus hexataenia',
 'scolopsis vosmeri',
 'pristipomoides zonatus',
 'bodianus unimaculatus',
 'pardachirus hedleyi',
 'serranocirrhitus latus',
 'sphyraena forsteri',
 'scolopsis affinis',
 'CUNWCB~Y ',
 'pseudanthias huchtii',
 'carangoides equula',
 'meuschenia scaber',
 'pseudanthias bicolor',
 'zeus faber',
 'lutjanus johnii',
 'pseudanthias pleurotaenia',
 'acanthaluteres vittiger',
 'pseudanthias squamipinnis',
 'latridopsis forsteri',
 'lethrinus erythracanthus',
 'lutjanus malabaricus',
 'scomberomorus commerson',
 'anampses caeruleopunctatus',
 'labroides dimi

In [0]:
config = det.DetConfig('fish', config_id)

In [0]:
config.BATCH_SIZE = 1
config.IMAGES_PER_GPU = 1
config.display()


Configurations:
ACTIVATION                     leakyrelu
ALL_CLASS_NAMES                ['BG', 'chascanopsetta lugubris', 'lutjanus semicinctus', 'pagrus auratus', 'cephalopholis microprion', 'pseudanthias lori', 'aprion virescens', 'stethojulis strigiventer', 'lutjanus fulviflamma', 'pentapodus paradiseus', 'halichoeres chloropterus', 'coris gaimard', 'epinephelus radiatus', 'scolopsis xenochrous', 'macropharyngodon negrosensis', 'alepes kleinii', 'amanses scopas', 'coris pictoides', 'pseudocheilinus hexataenia', 'scolopsis vosmeri', 'pristipomoides zonatus', 'bodianus unimaculatus', 'pardachirus hedleyi', 'serranocirrhitus latus', 'sphyraena forsteri', 'scolopsis affinis', 'CUNWCB~Y ', 'pseudanthias huchtii', 'carangoides equula', 'meuschenia scaber', 'pseudanthias bicolor', 'zeus faber', 'lutjanus johnii', 'pseudanthias pleurotaenia', 'acanthaluteres vittiger', 'pseudanthias squamipinnis', 'latridopsis forsteri', 'lethrinus erythracanthus', 'lutjanus malabaricus', 'scomberomorus co

In [0]:
# Create the model
model = modellib.MaskRCNN(mode='training', 
                          config=config,
                          model_dir=models_dir)

model.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
weights_path = os.path.join(models_dir, "mask_rcnn_coco.h5")
print("loading...", end='')
model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])
print("loaded.")

loading...loaded.


In [0]:
# Create the train and val dataset.
dataset_train, dataset_val = det.create_datasets_from_result(image_dir, config, result_masks)

# prepare datasets
dataset_train.prepare()
dataset_val.prepare()

Adding class   1:chascanopsetta lugubris
Adding class   2:lutjanus semicinctus
Adding class   3:pagrus auratus
Adding class   4:cephalopholis microprion
Adding class   5:pseudanthias lori
Adding class   6:aprion virescens
Adding class   7:stethojulis strigiventer
Adding class   8:lutjanus fulviflamma
Adding class   9:pentapodus paradiseus
Adding class  10:halichoeres chloropterus
Adding class  11:coris gaimard
Adding class  12:epinephelus radiatus
Adding class  13:scolopsis xenochrous
Adding class  14:macropharyngodon negrosensis
Adding class  15:alepes kleinii
Adding class  16:amanses scopas
Adding class  17:coris pictoides
Adding class  18:pseudocheilinus hexataenia
Adding class  19:scolopsis vosmeri
Adding class  20:pristipomoides zonatus
Adding class  21:bodianus unimaculatus
Adding class  22:pardachirus hedleyi
Adding class  23:serranocirrhitus latus
Adding class  24:sphyraena forsteri
Adding class  25:scolopsis affinis
Adding class  26:CUNWCB~Y 
Adding class  27:pseudanthias huch

In [0]:
# Experiment with training options.
# Since we're using a very small dataset, and starting from
# COCO trained weights, we don't need to train too long. Also,
# no need to train all layers, just the heads should do it.
print("Training all network")
history = model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=70,
            layers='all', 
            augmentation=seq_of_aug,
            )



Training all network

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/gdrive/My Drive/Mask_RCNN-master/fish_project/models/fish20190225T1254/mask_rcnn_fish_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (B

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/70
100/100 [==============================] - 335s 3s/step - loss: 2.3521 - rpn_class_loss: 0.0013 - rpn_bbox_loss: 0.2497 - mrcnn_class_loss: 0.6062 - mrcnn_bbox_loss: 0.6866 - mrcnn_mask_loss: 0.8083 - val_loss: 1.7187 - val_rpn_class_loss: 6.2587e-04 - val_rpn_bbox_loss: 0.2288 - val_mrcnn_class_loss: 0.2015 - val_mrcnn_bbox_loss: 0.5729 - val_mrcnn_mask_loss: 0.7149
Epoch 2/70
100/100 [==============================] - 234s 2s/step - loss: 1.7864 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.3102 - mrcnn_class_loss: 0.2401 - mrcnn_bbox_loss: 0.5409 - mrcnn_mask_loss: 0.6941 - val_loss: 2.1554 - val_rpn_class_loss: 0.0010 - val_rpn_bbox_loss: 0.8172 - val_mrcnn_class_loss: 0.1345 - val_mrcnn_bbox_loss: 0.5315 - val_mrcnn_mask_loss: 0.6712
Epoch 3/70
100/100 [==============================] - 230s 2s/step - loss: 1.7579 - rpn_class_loss: 6.9095e-04 - rpn_bbox_loss: 0.3122 - mrcnn_class_loss: 0.2413 - mrcnn_bbox_loss: 0.5227 - mrcnn_mask_loss: 0.6810 - val_loss: 1.5642 - val_rpn

ERROR:root:Error processing image {'id': 'epinephelus_ongus_3.jpg', 'source': 'fish', 'path': '/content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/raw_images/epinephelus_ongus_3.jpg'}
Traceback (most recent call last):
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1257, in load_image_gt
    assert mask.shape == mask_shape, "Augmentation shouldn't change mask size"
AssertionError: Augmentation shouldn't change mask size


 41/100 [===========>..................] - ETA: 1:56 - loss: 1.8170 - rpn_class_loss: 5.4862e-04 - rpn_bbox_loss: 0.2908 - mrcnn_class_loss: 0.2856 - mrcnn_bbox_loss: 0.5411 - mrcnn_mask_loss: 0.6989

...

100/100 [==============================] - 231s 2s/step - loss: 1.2500 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.1117 - mrcnn_class_loss: 0.1681 - mrcnn_bbox_loss: 0.3269 - mrcnn_mask_loss: 0.6422 - val_loss: 1.3461 - val_rpn_class_loss: 0.0014 - val_rpn_bbox_loss: 0.1918 - val_mrcnn_class_loss: 0.1650 - val_mrcnn_bbox_loss: 0.3500 - val_mrcnn_mask_loss: 0.6379
Epoch 39/70
100/100 [==============================] - 230s 2s/step - loss: 1.2464 - rpn_class_loss: 7.5745e-04 - rpn_bbox_loss: 0.0809 - mrcnn_class_loss: 0.1766 - mrcnn_bbox_loss: 0.3428 - mrcnn_mask_loss: 0.6454 - val_loss: 1.2087 - val_rpn_class_loss: 0.0014 - val_rpn_bbox_loss: 0.0659 - val_mrcnn_class_loss: 0.1501 - val_mrcnn_bbox_loss: 0.2941 - val_mrcnn_mask_loss: 0.6972
Epoch 40/70
100/100 [==============================] - 230s 2s/step - loss: 1.2883 - rpn_class_loss: 9.6634e-04 - rpn_bbox_loss: 0.0951 - mrcnn_class_loss: 0.1834 - mrcnn_bbox_loss: 0.3451 - mrcnn_mask_loss: 0.6636 - val_loss: 1.2820 - val_rpn_class_lo

ERROR:root:Error processing image {'id': 'bothus_mancus_4.jpg', 'source': 'fish', 'path': '/content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/raw_images/bothus_mancus_4.jpg'}
Traceback (most recent call last):
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1254, in load_image_gt
    hooks=imgaug.HooksImages(activator=hook))
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 407, in augment_image
    return self.augment_images([image], hooks=hooks)[0]
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 1956, in _augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_images
 

100/100 [==============================] - 232s 2s/step - loss: 1.0033 - rpn_class_loss: 9.2893e-04 - rpn_bbox_loss: 0.0802 - mrcnn_class_loss: 0.1376 - mrcnn_bbox_loss: 0.2618 - mrcnn_mask_loss: 0.5228 - val_loss: 0.6844 - val_rpn_class_loss: 8.2104e-04 - val_rpn_bbox_loss: 0.0642 - val_mrcnn_class_loss: 0.0960 - val_mrcnn_bbox_loss: 0.1997 - val_mrcnn_mask_loss: 0.3237
Epoch 57/70
  2/100 [..............................] - ETA: 3:17 - loss: 0.9033 - rpn_class_loss: 7.0215e-04 - rpn_bbox_loss: 0.0665 - mrcnn_class_loss: 0.1024 - mrcnn_bbox_loss: 0.2005 - mrcnn_mask_loss: 0.5332

ERROR:root:Error processing image {'id': 'bothus_mancus_4.jpg', 'source': 'fish', 'path': '/content/gdrive/My Drive/Mask_RCNN-master/fish_project/images/raw_images/bothus_mancus_4.jpg'}
Traceback (most recent call last):
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/gdrive/My Drive/Mask_RCNN-master/mrcnn/model.py", line 1254, in load_image_gt
    hooks=imgaug.HooksImages(activator=hook))
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 407, in augment_image
    return self.augment_images([image], hooks=hooks)[0]
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 1956, in _augment_images
    hooks=hooks
  File "/usr/local/lib/python3.6/dist-packages/imgaug/augmenters/meta.py", line 540, in augment_images
 

100/100 [==============================] - 232s 2s/step - loss: 0.8333 - rpn_class_loss: 0.0012 - rpn_bbox_loss: 0.0904 - mrcnn_class_loss: 0.1293 - mrcnn_bbox_loss: 0.2464 - mrcnn_mask_loss: 0.3660 - val_loss: 0.8159 - val_rpn_class_loss: 9.2737e-04 - val_rpn_bbox_loss: 0.0874 - val_mrcnn_class_loss: 0.1114 - val_mrcnn_bbox_loss: 0.2025 - val_mrcnn_mask_loss: 0.4137
Epoch 58/70
100/100 [==============================] - 232s 2s/step - loss: 0.8829 - rpn_class_loss: 8.2311e-04 - rpn_bbox_loss: 0.0664 - mrcnn_class_loss: 0.1374 - mrcnn_bbox_loss: 0.2495 - mrcnn_mask_loss: 0.4287 - val_loss: 1.0914 - val_rpn_class_loss: 9.9767e-04 - val_rpn_bbox_loss: 0.0435 - val_mrcnn_class_loss: 0.1418 - val_mrcnn_bbox_loss: 0.2470 - val_mrcnn_mask_loss: 0.6581
Epoch 59/70
100/100 [==============================] - 232s 2s/step - loss: 0.8356 - rpn_class_loss: 0.0012 - rpn_bbox_loss: 0.0780 - mrcnn_class_loss: 0.1473 - mrcnn_bbox_loss: 0.2562 - mrcnn_mask_loss: 0.3530 - val_loss: 0.7812 - val_rpn_clas

In [0]:
import math

def plot_history(history):
    
    fig = plt.figure(figsize=(16,10))
       
    n_history_loss = len(history.history)
    n_epochs = len(history.epoch)
    epoch = history.epoch
    
    # The loss is in pairs, one for train, one for val
    loss_stats = [ k for k in history.history.keys() if 'val_' not in k ]
    
    n_cols = 4
    n_rows = math.ceil(len(loss_stats) / n_cols) 
    
    for i, k in enumerate(loss_stats):
        val_k = 'val_'+k
        ax = plt.subplot( n_rows, n_cols, i+1)
        ax.plot(epoch, history.history[k], label=k)
        ax.plot(epoch, history.history[val_k], label=val_k)
        ax.set_xlabel('Epochs')
        ax.set_ylabel('Loss')
        ax.set_title(str(i)+' - '+k)
        plt.legend(shadow=True, fancybox=True)
    
    fig.tight_layout()
    plt.show()
    

In [0]:
# Plot training stats for each of the networks.
plot_history(history)